In [ ]:
import glob
import os
from pathlib import Path

import numpy as np
import xarray as xr

from fjordssim_notebooks.regrid import regrid_from_norkyst

In [ ]:
dirpath = os.path.join(Path.home(), "FjordsSim_data", "NorKyst")

In [ ]:
ds_grid = xr.open_dataset(
    os.path.join(Path.home(), "FjordsSim_data", "oslofjord", "OF_inner_105to232_bathymetry_v2.nc")
)

In [ ]:
lat_diff = np.diff(ds_grid["lat"].values)[0]
lat_faces = np.zeros(shape=ds_grid["lat"].values.shape[0] + 1)
lat_faces[0] = ds_grid["lat"].values[0] - lat_diff / 2
lat_faces[1:] = ds_grid["lat"].values + lat_diff / 2

In [ ]:
lon_diff = np.diff(ds_grid["lon"].values)[0]
lon_faces = np.zeros(shape=ds_grid["lon"].values.shape[0] + 1)
lon_faces[0] = ds_grid["lon"].values[0] - lon_diff / 2
lon_faces[1:] = ds_grid["lon"].values + lon_diff / 2

In [ ]:
z_faces = ds_grid.z_faces.values
z_centers = [(z_faces[i] + z_faces[i + 1]) / 2 for i in range(len(z_faces) - 1)]
ds_out_c = xr.Dataset(
    {
        "lat": (["lat"], ds_grid["lat"].values, {"units": "degrees_north"}),
        "lon": (["lon"], ds_grid["lon"].values, {"units": "degrees_east"}),
    }
)
ds_out_u = xr.Dataset(
    {
        "lat": (["lat"], ds_grid["lat"].values, {"units": "degrees_north"}),
        "lon": (["lon"], lon_faces, {"units": "degrees_east"}),
    }
)
ds_out_v = xr.Dataset(
    {
        "lat": (["lat"], lat_faces, {"units": "degrees_north"}),
        "lon": (["lon"], ds_grid["lon"].values, {"units": "degrees_east"}),
    }
)

In [ ]:
filepaths = sorted(glob.glob(os.path.join(dirpath, "*.nc")))

In [ ]:
time_list = []
temp_list = []
salt_list = []
u_list = []
v_list = []

for filepath in filepaths:
    ds_in = xr.open_dataset(filepath)
    np_time, np_temp, np_salt, np_u, np_v = regrid_from_norkyst(
        ds_in, ds_out_c, ds_out_u, ds_out_v, np.array(z_centers)
    )

    time_list.append(np_time)
    temp_list.append(np_temp)
    salt_list.append(np_salt)
    u_list.append(np_u)
    v_list.append(np_v)

In [ ]:
np_time = np.concatenate(time_list, axis=0)
np_temp = np.concatenate(temp_list, axis=0).astype(np.float32)
# np_temp = np.clip(np_temp, 0, 30)
np_salt = np.concatenate(salt_list, axis=0).astype(np.float32)
# np_salt = np.clip(np_salt, 0, 36)
np_u = np.concatenate(u_list, axis=0).astype(np.float32)
# np_u = np.clip(np_u, -1, 1)
np_v = np.concatenate(v_list, axis=0).astype(np.float32)
# np_v = np.clip(np_v, -1, 1)

#### Create output

In [ ]:
Tout_lambda = np.full(np_temp.shape, np.nan, dtype=np.float32)
Sout_lambda = np.full(np_salt.shape, np.nan, dtype=np.float32)
Uout_lambda = np.full(np_u.shape, np.nan, dtype=np.float32)
Vout_lambda = np.full(np_v.shape, np.nan, dtype=np.float32)

In [ ]:
Tout_lambda[:, :, :, :] = 1 / 60 / 60 / 24 / 20
Sout_lambda[:, :, :, :] = 1 / 60 / 60 / 24 / 20
Uout_lambda[:, :, :, :] = 1 / 60 / 60 / 24 / 20
Vout_lambda[:, :, :, :] = 1 / 60 / 60 / 24 / 20

In [ ]:
dsout = xr.Dataset(
    {
        "T": (["time", "Nz", "Ny", "Nx"], np_temp),
        "T_lambda": (["time", "Nz", "Ny", "Nx"], Tout_lambda),
        "S": (["time", "Nz", "Ny", "Nx"], np_salt),
        "S_lambda": (["time", "Nz", "Ny", "Nx"], Sout_lambda),
        "u": (["time", "Nz", "Ny", "Nx_faces"], np_u),
        "u_lambda": (["time", "Nz", "Ny", "Nx_faces"], Uout_lambda),
        "v": (["time", "Nz", "Ny_faces", "Nx"], np_v),
        "v_lambda": (["time", "Nz", "Ny_faces", "Nx"], Vout_lambda),
    },
    coords={
        "time": np_time,
        "Nz": z_centers,
        "Ny": ds_grid["lat"].values,
        "Ny_faces": lat_faces,
        "Nx": ds_grid["lon"].values,
        "Nx_faces": lon_faces,
    },
)

In [ ]:
dsout

In [ ]:
z_level = -1

In [ ]:
dsout.T.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.S.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.u.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.v.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout = dsout.fillna(-999)
dsout.to_netcdf(os.path.join(Path.home(), "FjordsSim_data", "oslofjord", "OF_inner_105to232_forcing.nc"))